In [2]:
import time
from PIL import Image
import numpy as np
from math import exp, sqrt
import os
from numba import njit, prange, set_num_threads, cuda
# from mpi4py import MPI

# Adjusted for script directory usage
# script_dir = os.path.dirname(os.path.realpath(__file__))

sobel_x = np.array([[-1, 0, 1],
                    [-2, 0, 2],
                    [-1, 0, 1]], dtype=np.int32)

sobel_y = np.array([[-1, -2, -1],
                    [ 0,  0,  0],
                    [ 1,  2,  1]], dtype=np.int32)

def read_image(path):
    try:
        return Image.open(path)
    except IOError as e:
        print(f"Error: {e}")
        return None

def write_image(image, path):
    create_outputs_directory_if_needed()
    try:
        image.save(path)
    except IOError as e:
        print(f"Error: {e}")

def create_outputs_directory_if_needed():
    outputs_path = "outputs"
    if not os.path.exists(outputs_path):
        os.makedirs(outputs_path)

def apply_gaussian_blur(image, kernel, radius, parallel=False):
    img_array = np.array(image)  # Convert PIL Image to NumPy array
    if parallel:
        blurred_img_array = apply_gaussian_blur_kernel_parallel(img_array, kernel, radius)
    else:
        blurred_img_array = apply_gaussian_blur_kernel_serial(img_array, kernel, radius)
    return Image.fromarray(blurred_img_array)  # Convert back to PIL Image

def apply_sobel(image, parallel=False):
    img_array = np.array(image.convert('L'))  # Convert to grayscale NumPy array
    if parallel:
        edges_img_array = apply_sobel_filter_parallel(img_array)
    else:
        edges_img_array = apply_sobel_filter_serial(img_array)
    return Image.fromarray(edges_img_array).convert('RGB')  # Convert back to PIL Image in RGB

def generate_gaussian_blur_kernel(radius, sigma):
    size = 2 * radius + 1
    kernel = np.zeros((size, size))
    sum_val = 0.0

    for i in range(-radius, radius + 1):
        for j in range(-radius, radius + 1):
            kernel[i + radius, j + radius] = exp(-(i**2 + j**2) / (2 * sigma**2))
            sum_val += kernel[i + radius, j + radius]

    kernel /= sum_val
    return kernel

def apply_gaussian_blur_kernel_serial(img_array, kernel, radius):
    height, width = img_array.shape[:2]
    padded_img = np.pad(img_array, ((radius, radius), (radius, radius), (0, 0)), 'edge')
    blurred_img = np.zeros_like(img_array)

    for y in range(height):
        for x in range(width):
            for c in range(3):  # For each color channel
                acc = np.sum(kernel * padded_img[y:y+2*radius+1, x:x+2*radius+1, c])
                blurred_img[y, x, c] = min(max(int(acc), 0), 255)

    return blurred_img

def apply_sobel_filter_serial(img_array):
    height, width = img_array.shape
    edges_img = np.zeros_like(img_array)

    for y in range(1, height-1):
        for x in range(1, width-1):
            gx = np.sum(sobel_x * img_array[y-1:y+2, x-1:x+2])
            gy = np.sum(sobel_y * img_array[y-1:y+2, x-1:x+2])
            edges_img[y, x] = min(sqrt(gx**2 + gy**2), 255)

    return edges_img

@njit(parallel=True)
def apply_gaussian_blur_kernel_parallel(img_array, kernel, radius):
    height, width = img_array.shape[:2]
    blurred_img = np.zeros_like(img_array)

    for y in prange(height):
        for x in prange(width):
            for c in range(3):  # For each color channel
                sum_val = 0.0
                for ky in range(-radius, radius + 1):
                    for kx in range(-radius, radius + 1):
                        px = min(max(x + kx, 0), width - 1)
                        py = min(max(y + ky, 0), height - 1)
                        sum_val += img_array[py, px, c] * kernel[ky + radius, kx + radius]
                blurred_img[y, x, c] = sum_val
    return blurred_img

@njit(parallel=True)
def apply_sobel_filter_parallel(img_array):
    height, width = img_array.shape
    edges_img = np.zeros((height, width), np.uint8)

    for y in prange(1, height-1):
        for x in prange(1, width-1):
            gx = 0.0
            gy = 0.0
            # Apply the Sobel kernel to the grayscale image
            for ky in range(-1, 2):
                for kx in range(-1, 2):
                    val = img_array[y + ky, x + kx]
                    gx += val * sobel_x[ky + 1, kx + 1]
                    gy += val * sobel_y[ky + 1, kx + 1]
            magnitude = int(min(np.sqrt(gx**2 + gy**2), 255))
            edges_img[y, x] = magnitude
    return edges_img

def apply_gaussian_blur_chunk(img_array, kernel, radius):
    height, width = img_array.shape[:2]
    blurred_img = np.zeros_like(img_array)
    for y in range(height):
        for x in range(width):
            for c in range(3):
                sum_val = 0.0
                for ky in range(-radius, radius + 1):
                    for kx in range(-radius, radius + 1):
                        py = min(max(y + ky, 0), height - 1)
                        px = min(max(x + kx, 0), width - 1)
                        sum_val += img_array[py, px, c] * kernel[ky + radius, kx + radius]
                blurred_img[y, x, c] = sum_val
    return blurred_img

def apply_sobel_chunk(img_array):
    height, width = img_array.shape
    edges_img = np.zeros((height, width), np.uint8)
    for y in range(1, height - 1):
        for x in range(1, width - 1):
            gx = np.sum(sobel_x * img_array[y - 1:y + 2, x - 1:x + 2])
            gy = np.sum(sobel_y * img_array[y - 1:y + 2, x - 1:x + 2])
            magnitude = int(min(np.sqrt(gx**2 + gy**2), 255))
            edges_img[y, x] = magnitude
    return edges_img

@cuda.jit
def apply_gaussian_blur_kernel_cuda(img_array, kernel, radius, blurred_img):
    y, x = cuda.grid(2)
    height, width = img_array.shape[:2]
    if y < height and x < width:
        for c in range(3):  # For each color channel
            sum_val = 0.0
            for ky in range(-radius, radius + 1):
                for kx in range(-radius, radius + 1):
                    py = min(max(y + ky, 0), height - 1)
                    px = min(max(x + kx, 0), width - 1)
                    sum_val += img_array[py, px, c] * kernel[ky + radius, kx + radius]
            blurred_img[y, x, c] = sum_val

def apply_gaussian_blur_cuda(img_array, kernel, radius):
    # Allocate output array
    blurred_img = np.zeros_like(img_array)

    # Copy data to device
    d_img_array = cuda.to_device(img_array)
    d_kernel = cuda.to_device(kernel)
    d_blurred_img = cuda.to_device(blurred_img)

    # Setup the grid and blocks
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(img_array.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(img_array.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    # Launch kernel
    apply_gaussian_blur_kernel_cuda[blockspergrid, threadsperblock](d_img_array, d_kernel, radius, d_blurred_img)

    # Copy result back to host
    d_blurred_img.copy_to_host(blurred_img)

    return blurred_img

@cuda.jit
def apply_sobel_filter_kernel_cuda(img_array, sobel_x, sobel_y, edges_img):
    y, x = cuda.grid(2)
    height, width = img_array.shape[:2]
    if y < height and x < width:
        gx = 0.0
        gy = 0.0
        for ky in range(-1, 2):
            for kx in range(-1, 2):
                val = img_array[y + ky, x + kx]
                gx += val * sobel_x[ky + 1, kx + 1]
                gy += val * sobel_y[ky + 1, kx + 1]
        magnitude = int(min(sqrt(gx**2 + gy**2), 255))
        edges_img[y, x] = magnitude

def apply_sobel_filter_cuda(img_array):
    height, width = img_array.shape
    edges_img = np.zeros_like(img_array)

    # Copy data to device
    d_img_array = cuda.to_device(img_array)
    d_edges_img = cuda.to_device(edges_img)

    # Setup the grid and blocks
    threadsperblock = (16, 16)
    blockspergrid_x = int(np.ceil(img_array.shape[0] / threadsperblock[0]))
    blockspergrid_y = int(np.ceil(img_array.shape[1] / threadsperblock[1]))
    blockspergrid = (blockspergrid_x, blockspergrid_y)

    # Launch kernel
    apply_sobel_filter_kernel_cuda[blockspergrid, threadsperblock](d_img_array, sobel_x, sobel_y, d_edges_img)

    # Copy result back to host
    d_edges_img.copy_to_host(edges_img)

    return edges_img

# Remaining utility functions and __main__ block remain unchanged

# Example __main__ modifications for CUDA
if __name__ == "__main__":
    # Adjust the following variables as needed
    file_name = "squidward_painting"
    file_extension = "jpg"
    input_image_path = f"assets/{file_name}.{file_extension}"
    output_blurred_path = f"outputs/{file_name}_blurred_cuda.{file_extension}"
    output_edges_path = f"outputs/{file_name}_sobel_cuda.{file_extension}"

    radius = 9
    sigma = 20.0

    # CUDA Example
    input_image = read_image(input_image_path)
    if input_image is not None:
        kernel = generate_gaussian_blur_kernel(radius, sigma)
        img_array = np.array(input_image)
        blurred_img_array = apply_gaussian_blur_cuda(img_array, kernel, radius)
        blurred_image = Image.fromarray(blurred_img_array.astype('uint8'), 'RGB')

        write_image(blurred_image, output_blurred_path)

        img_array = np.array(input_image.convert('L'))
        edges_img_array = apply_sobel_filter_cuda(img_array)
        edges_image = Image.fromarray(edges_img_array, 'L').convert('RGB')
        write_image(edges_image, output_edges_path)


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:886: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))
